## Import Libraries

In [1]:
import gc
import os
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import math
from subprocess import check_output
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.impute import SimpleImputer 
from sklearn.impute import MissingIndicator
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, GroupShuffleSplit
#from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import FeatureUnion
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import reciprocal, uniform, kurtosis, skew

#warnings.filterwarnings('ignore')

## Read in Data (.csv)

In [2]:
train = pd.read_csv('X_train.csv')
test = pd.read_csv('X_test.csv')
y = pd.read_csv('y_train.csv')

## Feature Engineering 

This set of functions for feature engineering is taken from https://www.kaggle.com/jesucristo/1-smart-robots-most-complete-notebook. At first, I will simply use the functions to prepare my dataset. Later, I will see if I can add them to my pipeline.

In [3]:
# https://stackoverflow.com/questions/53033620/how-to-convert-euler-angles-to-quaternions-and-get-the-same-euler-angles-back-fr?rq=1
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

def fe_step0 (actual):
    
    # https://www.mathworks.com/help/aeroblks/quaternionnorm.html
    # https://www.mathworks.com/help/aeroblks/quaternionmodulus.html
    # https://www.mathworks.com/help/aeroblks/quaternionnormalize.html
    
    # Spoiler: you don't need this ;)
    
    actual['norm_quat'] = (actual['orientation_X']**2 + actual['orientation_Y']**2 + actual['orientation_Z']**2 + actual['orientation_W']**2)
    actual['mod_quat'] = (actual['norm_quat'])**0.5
    actual['norm_X'] = actual['orientation_X'] / actual['mod_quat']
    actual['norm_Y'] = actual['orientation_Y'] / actual['mod_quat']
    actual['norm_Z'] = actual['orientation_Z'] / actual['mod_quat']
    actual['norm_W'] = actual['orientation_W'] / actual['mod_quat']
    
    return actual

def fe_step1 (actual):
    """Quaternions to Euler Angles"""
    
    x, y, z, w = actual['norm_X'].tolist(), actual['norm_Y'].tolist(), actual['norm_Z'].tolist(), actual['norm_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    actual['euler_x'] = nx
    actual['euler_y'] = ny
    actual['euler_z'] = nz
    
    return actual

def feat_eng(data):
    
    df = pd.DataFrame()
    data['totl_anglr_vel'] = (data['angular_velocity_X']**2 + data['angular_velocity_Y']**2 + data['angular_velocity_Z']**2)** 0.5
    data['totl_linr_acc'] = (data['linear_acceleration_X']**2 + data['linear_acceleration_Y']**2 + data['linear_acceleration_Z']**2)**0.5
    data['totl_xyz'] = (data['orientation_X']**2 + data['orientation_Y']**2 + data['orientation_Z']**2)**0.5
    data['acc_vs_vel'] = data['totl_linr_acc'] / data['totl_anglr_vel']
    
    def mean_change_of_abs_change(x):
        return np.mean(np.diff(np.abs(np.diff(x))))
    
    for col in data.columns:
        if col in ['row_id','series_id','measurement_number']:
            continue
        df[col + '_mean'] = data.groupby(['series_id'])[col].mean()
        df[col + '_median'] = data.groupby(['series_id'])[col].median()
        df[col + '_max'] = data.groupby(['series_id'])[col].max()
        df[col + '_min'] = data.groupby(['series_id'])[col].min()
        df[col + '_std'] = data.groupby(['series_id'])[col].std()
        df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        df[col + '_mean_abs_chg'] = data.groupby(['series_id'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df[col + '_mean_change_of_abs_change'] = data.groupby('series_id')[col].apply(mean_change_of_abs_change)
        df[col + '_abs_max'] = data.groupby(['series_id'])[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby(['series_id'])[col].apply(lambda x: np.min(np.abs(x)))
        df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2
        
    return df


### Statistical Features

In [4]:
def _kurtosis(x):
    return kurtosis(x)

def CPT5(x):
    den = len(x)*np.exp(np.std(x))
    return sum(np.exp(x))/den

def skewness(x):
    return skew(x)

def SSC(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    xn_i1 = x[0:len(x)-2]  # xn-1
    ans = np.heaviside((xn-xn_i1)*(xn-xn_i2),0)
    return sum(ans[1:]) 

def wave_length(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    return sum(abs(xn_i2-xn))
    
def norm_entropy(x):
    tresh = 3
    return sum(np.power(abs(x),tresh))

def SRAV(x):    
    SRA = sum(np.sqrt(abs(x)))
    return np.power(SRA/len(x),2)

def mean_abs(x):
    return sum(abs(x))/len(x)

def zero_crossing(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1
    return sum(np.heaviside(-xn*xn_i2,0))

### Advanced Features?

In [5]:
def fe_advanced_stats(data):
    
    df = pd.DataFrame()
    
    for col in data.columns:
        if col in ['row_id','series_id','measurement_number']:
            continue
        if 'orientation' in col:
            continue
            
        print ("FE on column ", col, "...")
        
        df[col + '_skew'] = data.groupby(['series_id'])[col].skew()
        df[col + '_mad'] = data.groupby(['series_id'])[col].mad()
        df[col + '_q25'] = data.groupby(['series_id'])[col].quantile(0.25)
        df[col + '_q75'] = data.groupby(['series_id'])[col].quantile(0.75)
        df[col + '_q95'] = data.groupby(['series_id'])[col].quantile(0.95)
        df[col + '_iqr'] = df[col + '_q75'] - df[col + '_q25']
        df[col + '_CPT5'] = data.groupby(['series_id'])[col].apply(CPT5) 
        df[col + '_SSC'] = data.groupby(['series_id'])[col].apply(SSC) 
        df[col + '_skewness'] = data.groupby(['series_id'])[col].apply(skewness)
        df[col + '_wave_lenght'] = data.groupby(['series_id'])[col].apply(wave_length)
        df[col + '_norm_entropy'] = data.groupby(['series_id'])[col].apply(norm_entropy)
        df[col + '_SRAV'] = data.groupby(['series_id'])[col].apply(SRAV)
        df[col + '_kurtosis'] = data.groupby(['series_id'])[col].apply(_kurtosis) 
        df[col + '_zero_crossing'] = data.groupby(['series_id'])[col].apply(zero_crossing) 
        
    return df

### Create Features

In [7]:
# This soooo needs a pipeline
# train
data = train

data = fe_step0(data)
data = fe_step1(data)
data = feat_eng(data)
data = fe_advanced_stats(data)

In [11]:
# test
test = fe_step0(test)
test = fe_step1(test)
test = feat_eng(test)
test = fe_advanced_stats(test)

FE on column  angular_velocity_X_mean ...
FE on column  angular_velocity_X_median ...
FE on column  angular_velocity_X_max ...
FE on column  angular_velocity_X_min ...
FE on column  angular_velocity_X_std ...
FE on column  angular_velocity_X_range ...
FE on column  angular_velocity_X_maxtoMin ...
FE on column  angular_velocity_X_mean_abs_chg ...
FE on column  angular_velocity_X_mean_change_of_abs_change ...
FE on column  angular_velocity_X_abs_max ...
FE on column  angular_velocity_X_abs_min ...
FE on column  angular_velocity_X_abs_avg ...
FE on column  angular_velocity_Y_mean ...
FE on column  angular_velocity_Y_median ...
FE on column  angular_velocity_Y_max ...
FE on column  angular_velocity_Y_min ...
FE on column  angular_velocity_Y_std ...
FE on column  angular_velocity_Y_range ...
FE on column  angular_velocity_Y_maxtoMin ...
FE on column  angular_velocity_Y_mean_abs_chg ...
FE on column  angular_velocity_Y_mean_change_of_abs_change ...
FE on column  angular_velocity_Y_abs_max ..

/home/gopherguy14/anaconda3/envs/kaggle_comps/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


FE on column  angular_velocity_Z_mean_abs_chg ...
FE on column  angular_velocity_Z_mean_change_of_abs_change ...
FE on column  angular_velocity_Z_abs_max ...
FE on column  angular_velocity_Z_abs_min ...
FE on column  angular_velocity_Z_abs_avg ...
FE on column  linear_acceleration_X_mean ...
FE on column  linear_acceleration_X_median ...
FE on column  linear_acceleration_X_max ...
FE on column  linear_acceleration_X_min ...
FE on column  linear_acceleration_X_std ...
FE on column  linear_acceleration_X_range ...
FE on column  linear_acceleration_X_maxtoMin ...
FE on column  linear_acceleration_X_mean_abs_chg ...
FE on column  linear_acceleration_X_mean_change_of_abs_change ...
FE on column  linear_acceleration_X_abs_max ...
FE on column  linear_acceleration_X_abs_min ...
FE on column  linear_acceleration_X_abs_avg ...
FE on column  linear_acceleration_Y_mean ...
FE on column  linear_acceleration_Y_median ...
FE on column  linear_acceleration_Y_max ...
FE on column  linear_acceleration_

## Save as pickle

In [12]:
data.to_pickle("train_features.pkl")
test.to_pickle("test_features.pkl")

## Read-in pickle

In [14]:
train_feat = pd.read_pickle("train_features.pkl")
test_feat = pd.read_pickle("test_features.pkl")

## Fix Zeros / NaN

In [19]:
## replace all 0 with Nan
train_feat.replace(0, np.nan, inplace=True)
train_feat.replace(-np.inf, np.nan, inplace=True)
train_feat.replace(np.inf, np.nan, inplace=True)
test_feat.replace(0, np.nan, inplace=True)
test_feat.replace(-np.inf, np.nan, inplace=True)
test_feat.replace(np.inf, np.nan, inplace=True)

## Make list of columns which are all NaN
def drop_missing(df):
    p = ( df.isnull().sum() / df.isnull().count() ).sort_values(ascending=False)
    keep_columns = list( p[ p < 0.06 ].index )
    
    return keep_columns

train_col_to_keep = drop_missing(train_feat)
test_col_to_keep = drop_missing(test_feat)
master_col_to_keep = set(train_col_to_keep) & set(test_col_to_keep)

final_train = train_feat[master_col_to_keep]
final_test = test_feat[master_col_to_keep]

## Save Final Features to Pickle

In [22]:
final_train.to_pickle("final_train_features.pkl")
final_test.to_pickle("final_test_features.pkl")

## Make Subset

In [23]:
final_train = final_train.set_index('series_id').join(y.set_index('series_id'))
trainb = final_train.loc[final_train['group_id'].isin([2,13,23,37,49])]

KeyError: 'series_id'

## Encode the Reponse

In [ ]:
encoded_response=pd.DataFrame()
instance_LabelEncoder = LabelEncoder()
encoded_response['surface'] = instance_LabelEncoder.fit_transform(trainb['surface'])
y_data = encoded_response['surface']

## Pipeline

In [ ]:
parameters = {
    'max_depth': [12,15],
    'n_estimators': [5, 10, 15],
    'min_samples_leaf': [2,4]
}
             
## Add summary transforms (calculate the mean, stdev, max, median, min) per group

## Try a different cross validation method


class_pipe = Pipeline([
    ('smote', SMOTE(random_state=42, sampling_strategy='minority')),
    # ('add_features', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ('standardize', StandardScaler()),
    ('GrdSrch', GridSearchCV(RandomForestClassifier(), param_grid=parameters, scoring='accuracy'))
])

In [ ]:
# Define addtional variables in hopes of increasing readability
X_data = trainb[['orientation_X','orientation_Y','orientation_Z','orientation_W','angular_velocity_X','angular_velocity_Y','angular_velocity_Z','linear_acceleration_X','linear_acceleration_Y','linear_acceleration_Z']]
grp_data = trainb['group_id']

In [ ]:
## I think this will work
name = ["Random Forest Classifier"]

model_list = {}
## Attach StratifiedKFold or whatever to CV
#GrdSrch = GridSearchCV(class_pipe, parameters, verbose=2, scoring='accuracy')
gs_classif = class_pipe.fit(X_data, y_data)
    
## Use cross_val_score
score = cross_val_score(gs_classif, X=X_data, y=y_data, cv=GroupShuffleSplit().split(X=X_data, y=y_data, groups=grp_data))
print("{} score: {}".format(name, score.mean()))

In [ ]:
y_data = encoded_response['surface']

In [ ]:
gs_classif.named_steps['GrdSrch'].best_estimator_

In [ ]:
## Generate Predictions
y_hat = gs_classif.best_estimator_.predict(test[['orientation_X','orientation_Y','orientation_Z','orientation_W', \
                                                        'angular_velocity_X','angular_velocity_Y','angular_velocity_Z', \
                                                        'linear_acceleration_X','linear_acceleration_Y','linear_acceleration_Z']])

## Transform back to labels
test['surface'] = instance_LabelEncoder.inverse_transform(y_hat)

## Try again with 250 estimators

In [ ]:
encoded_response=pd.DataFrame()
instance_LabelEncoder = LabelEncoder()
encoded_response['surface'] = instance_LabelEncoder.fit_transform(train['surface'])
y_data = encoded_response['surface']
X_data = train[['orientation_X','orientation_Y','orientation_Z','orientation_W','angular_velocity_X','angular_velocity_Y','angular_velocity_Z','linear_acceleration_X','linear_acceleration_Y','linear_acceleration_Z']]


## Final Pipe
best_pipe = Pipeline([
    ('smote', SMOTE(random_state=42, sampling_strategy='minority')),
    ('add_features', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ('standardize', StandardScaler()),
    ('classify', RandomForestClassifier(max_depth=15, min_samples_leaf=2, min_samples_split=2, n_estimators=250))
])

# Run random forest on everything
best_pipe.fit(X_data, y_data)



In [ ]:
yhat = best_pipe.predict(test[['orientation_X','orientation_Y','orientation_Z','orientation_W', \
                                                        'angular_velocity_X','angular_velocity_Y','angular_velocity_Z', \
                                                        'linear_acceleration_X','linear_acceleration_Y','linear_acceleration_Z']])

In [ ]:
## Transform back to labels
test['surface'] = instance_LabelEncoder.inverse_transform(y_hat)

In [ ]:
## join to submission file
submission = pd.read_csv('sample_submission.csv')
answers = test.groupby('series_id').first()[['surface']]
submission['surface'] = answers['surface']

## save as csv
submission.to_csv('submission_11Apr2019_djs.csv', index=False)